# 8장. 핵심 키워드 추출 (Keyword Extraction)

# 8-0 데이터 준비

# 8-1 TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용


In [5]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
  res = requests.get(url)
  bs = BeautifulSoup(res.content, 'html.parser')

  title = bs.select('h3#articleTitle')[0].text #제목
  content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ") #본문
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
  return  content.strip()

docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

5

### 1) 전처리

In [7]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath='C:\mecab\mecab-ko-dic')

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_df=0.85, max_features=10000)
word_count_vector = count_vectorizer.fit_transform(preprocessed_docs)
list(count_vectorizer.vocabulary_.keys())[:10]

['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '구축', '민간']

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

### 3) 핵심키워드 추출

In [10]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [11]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vectorizer.get_feature_names() # TF-IDF 단어 목록
tf_idf_vector = tfidf_transformer.transform(count_vectorizer.transform([doc])) # 문서의 tf-idf 추출
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬
 
# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)
 
print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


===== 원문 =====
과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자

=== 핵심키워드 ===
('플랫', 0.2526148007071733)
('계획', 0.21652697203472)
('정통부', 0.18043914336226666)
('과기', 0.18043914336226666)
('통해', 0.17469259767293158)


In [12]:
tf_idf_vector.tocoo().data

array([0.03608783, 0.03608783, 0.05823087, 0.07217566, 0.03608783,
       0.03608783, 0.03608783, 0.02911543, 0.03608783, 0.14557716,
       0.03608783, 0.03608783, 0.02911543, 0.02911543, 0.07217566,
       0.11646173, 0.03608783, 0.03608783, 0.04833688, 0.03608783,
       0.03608783, 0.2526148 , 0.03608783, 0.02911543, 0.03608783,
       0.02911543, 0.02911543, 0.03608783, 0.10826349, 0.1746926 ,
       0.07217566, 0.03608783, 0.10826349, 0.03608783, 0.07217566,
       0.03608783, 0.02911543, 0.02911543, 0.07217566, 0.03608783,
       0.04066251, 0.02911543, 0.07217566, 0.03608783, 0.03608783,
       0.10826349, 0.03608783, 0.03608783, 0.10826349, 0.03608783,
       0.03608783, 0.02911543, 0.03608783, 0.03608783, 0.02911543,
       0.02911543, 0.03608783, 0.03608783, 0.10826349, 0.02911543,
       0.07217566, 0.03608783, 0.02911543, 0.03608783, 0.1208422 ,
       0.07217566, 0.18043914, 0.02911543, 0.04066251, 0.02911543,
       0.09667376, 0.07217566, 0.03608783, 0.02033125, 0.02911

## 실습 2. gensim 활용


### 1) 전처리

In [20]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath = 'C:\mecab\mecab-ko-dic')

preprocessed_docs = []
for doc in docs :
  # 명사와 동사만으로 문서 전처리
  preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [14]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(valuue) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

### 3) 핵심키워드 추출

In [15]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [16]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


=== 핵심키워드 ===
('플랫', 0.260111262735105)
('폼', 0.260111262735105)
('계획', 0.2229525109158043)
('정통부', 0.18579375909650356)
('과기', 0.18579375909650356)


# 8-2 Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 실습 1. 행렬 활용 


### 1) 토큰화 (Tokenization)

분석 텍스트 정제

### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [165]:
token = ['딸기', '바나나', '사과', '딸기', '파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

vocab2idx = {val:idx for idx, val in enumerate(vocab)} #vocab을 인덱스로 변환
idx2vocab = dict(enumerate(vocab))  #인덱스를 vocab으로 변환
vocab2idx

{'바나나': 0, '사과': 1, '파인애플': 2, '딸기': 3}

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [166]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len))

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((1,vocab_len))[0]
print(score)
# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2
covered_coocurrences = np.zeros_like(weighted_edge)
# weighted_edge 구현
for i in range(len(token)-(window_size-1)):
    covered_coocurrences[vocab2idx[token[i:i+window_size][0]],vocab2idx[token[i:i+window_size][1]]]=1
    covered_coocurrences[vocab2idx[token[i:i+window_size][1]],vocab2idx[token[i:i+window_size][0]]]=1
    
covered_coocurrences

weighted_edge = np.transpose(covered_coocurrences*(score/sum(covered_coocurrences)))
print(weighted_edge)

[1. 1. 1. 1.]
[[0.         0.5        0.         0.5       ]
 [0.5        0.         0.         0.5       ]
 [0.         0.         0.         1.        ]
 [0.33333333 0.33333333 0.33333333 0.        ]]


### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [167]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):    
    prev_score = np.copy(score)
    score = (1-d)+d*np.dot(score/sum(covered_coocurrences),covered_coocurrences)
    if max(abs(prev_score-score))<threshold:
        break
    pass
score

array([0.98370451, 0.98370451, 0.56562258, 1.46696839])

In [168]:
score_tuple=[(val, i) for i,val in enumerate(score)]
sorted(score_tuple,reverse=True)
score_tuple[0][0]

0.9837045132067815

### 5) 핵심 단어 추출

In [171]:
# score_tuple = [(val, i) for i,val in enumerate(score)]
# sorted_score_tuple = sorted(score_tuple,reverse=True)
# sorted_index = [val[1] for val in sorted_score_tuple]
# print(sorted_index)
print(np.argsort(score)[::-1])
idx_score = np.argsort(score)[::-1]
n = 4

print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(idx2vocab[idx_score[i]])+" : " + str(score[idx_score][i]))

[3 1 0 2]

=== 핵심키워드 ===
딸기 : 1.4669683910772888
사과 : 0.9837045132067815
바나나 : 0.9837045132067815
파인애플 : 0.5656225825091485


In [97]:
sorted_index[0]

3

## 뉴스기사 실습

In [3]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
    res = requests.get(url)
    bs = BeautifulSoup(res.content, 'html.parser')

    title = bs.select('h3#articleTitle')[0].text #제목
    content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ") #본문
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    return  content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:50]

'과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 510'

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [173]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath = 'C:\mecab\mecab-ko-dic')

nodes = [token for token in mecab.pos(doc) if token[1] in ['NNG','NNP']] #NNG, NNP를 스코어 계산 대상(노드)로 제한
tokens = [token for token in mecab.pos(doc)] #탐색할 토큰 전체

In [174]:
token = [val[0] for val in tokens]
vocab = list(set([val[0] for val in nodes]))
print(token)
vocab2idx = {val:idx for idx, val in enumerate(vocab)} #vocab을 인덱스로 변환
idx2vocab = dict(enumerate(vocab))  #인덱스를 vocab으로 변환

['과기', '정통부', ',', '22', '일', '유영민', '장관', '등', '참석', '해', '기념행사', '2021', '년', '까지', '1516', '억', '원', '투입', ',', '5100', '여종', '데이터', '구축', '민간', '클라우드', '통한', '외부', '연계', '체계', '도', '.', '."', '개방', '성', '강화', '"[', '이', '데일리', '이재운', '기자', ']', '국가', '차원', '의', '빅', '데이터', '활용', '시대', '가', '열린다', '.', '새로운', '산업', '창출', '과', '기존', '산업', '의', '변화', '에', '이르', '는', '‘', '혁신', '성', '장', '’', '을', '위한', '센터', '가', '문', '을', '연다', '.', '10', '개', '분야', '에', '걸쳐', '‘', '데이터', '경제', '’', '의', '발전', '을', '위한', '정부', '의', '청사진', '을', '현실', '로', '구현', '하', '는', '데', '앞장선다는', '계획', '이', '다', '.', '22', '일', '과학', '기술', '정보', '통신부', '는', '서울', '중구', '대한', '상공', '회의소', '에서', '데이터', '생태', '계', '조성', '과', '혁신', '성장', '의', '기반', '마련', '을', '위한', '‘', '빅', '데이터', '플랫', '폼', '및', '센터', '’', '출범식', '행사', '를', '개최', '했', '다', '.', '유영민', '과기', '정통부', '장관', '을', '비롯', '해', '노웅래', '국회', '과학', '기술', '정보', '방송', '통신', '위원회', '위원장', '등', '300', '여', '명', '이', '참가', '했', '다', '.', '◇', '10', '개', '분야', '100

### 2) 그래프 생성 (weighted edge 계산)

In [175]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len))

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((1,vocab_len))[0]
print(len(score))
# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 5
covered_coocurrences = np.zeros_like(weighted_edge)
# weighted_edge 구현
for i in range(len(token)-(window_size-1)):
    if token[i] in vocab:
        if i<window_size-1:
            win_token = token[:i+window_size]
            win_token.remove(token[i])
            for tok in win_token:m
                if tok in vocab:
                    covered_coocurrences[vocab2idx[tok], vocab2idx[token[i]]] = 1
                    covered_coocurrences[vocab2idx[token[i]], vocab2idx[tok]] = 1
        else :
            win_token = token[i-window_size+1:i+window_size]
            win_token.remove(token[i])
            for tok in win_token:
                if tok in vocab:
                    covered_coocurrences[vocab2idx[tok], vocab2idx[token[i]]] = 1
                    covered_coocurrences[vocab2idx[token[i]], vocab2idx[tok]] = 1
    
covered_coocurrences
sum(covered_coocurrences)
weighted_edge = np.transpose(covered_coocurrences*(score/sum(covered_coocurrences)))
print(weighted_edge)
print(covered_coocurrences)

252
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.11111111 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.5        0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\YOON\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in multiply


### 3) 각 노드의 score계산

In [160]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):    
    prev_score = np.copy(score)
    score = (1-d)+d*np.dot(score/sum(covered_coocurrences),covered_coocurrences)
    if max(abs(prev_score-score))<threshold:
        break
    pass
score

array([ 0.60446443,  1.41890499,  0.64994479,  0.99759924,  0.54424674,
        0.67958754,  1.65060301,  0.65077514,  0.79874191,  0.78167425,
        1.58359585,  0.75441982,  0.62870625,  0.80221281,  1.06170421,
        0.73968052,  1.09337066,  1.03473799,  0.47964696,  1.28472265,
        0.56716512,  1.22223461,  1.2007146 ,  2.19615374,  3.72266108,
        1.86461649,  0.96531607,  0.66978285,  0.69409716,  0.72506818,
        0.88819966,  0.68458884,  0.83363846,  0.50527746,  0.80825163,
        1.09899316,  0.74154347,  1.34306888,  0.46671696,  0.66290582,
        0.62959279,  0.63357929,  1.13927448,  0.40899153,  0.56214676,
        0.5707823 ,  0.7546089 ,  1.10784609,  0.68056267,  0.86921259,
        1.70227683,  0.60369385,  0.50919907,  1.40382825,  0.41222349,
        0.59568804,  0.84509021,  0.65827062,  0.73637297,  1.42938114,
        0.97459798,  0.68560789,  0.75441982,  0.9732938 ,  0.70356767,
        0.87785458,  0.72131239,  2.56182766,  0.85206868,  1.17

In [161]:
score_tuple=[(val, i) for i,val in enumerate(score)]
sorted(score_tuple,reverse=True)
score_tuple[0][0]

0.6044644307848853

### 4)핵심 단어 추출

In [163]:
score_tuple = [(val, i) for i,val in enumerate(score)]
sorted_score_tuple = sorted(score_tuple,reverse=True)
sorted_index = [val[1] for val in sorted_score_tuple]
print(sorted_index)
n = 10

print("\n=== 핵심키워드 ===")
for i in range(0,n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))

[165, 248, 24, 114, 182, 74, 94, 119, 67, 221, 109, 23, 136, 131, 121, 99, 152, 118, 25, 84, 96, 175, 50, 78, 82, 216, 6, 93, 10, 130, 156, 73, 59, 1, 53, 141, 198, 92, 37, 112, 137, 154, 179, 195, 228, 226, 19, 244, 21, 22, 69, 240, 155, 177, 196, 233, 88, 42, 241, 162, 47, 157, 35, 16, 77, 246, 14, 239, 123, 17, 86, 164, 3, 148, 212, 100, 176, 60, 172, 63, 209, 247, 90, 26, 71, 110, 206, 225, 76, 120, 108, 30, 65, 49, 197, 116, 68, 223, 222, 192, 183, 80, 205, 134, 56, 166, 214, 32, 145, 149, 249, 202, 187, 203, 75, 87, 102, 106, 168, 83, 138, 81, 34, 13, 8, 173, 227, 111, 95, 9, 101, 218, 132, 89, 98, 220, 207, 135, 107, 146, 181, 46, 62, 11, 229, 161, 153, 36, 79, 15, 58, 186, 103, 217, 158, 29, 66, 184, 251, 210, 215, 167, 64, 140, 28, 236, 189, 163, 204, 200, 61, 31, 85, 48, 5, 104, 142, 199, 150, 27, 235, 91, 171, 39, 125, 237, 191, 144, 57, 194, 7, 2, 160, 201, 129, 41, 169, 238, 105, 40, 133, 12, 139, 113, 232, 115, 151, 124, 0, 51, 55, 224, 159, 250, 188, 170, 234, 97, 174, 2

### networks 패키지 이용하는 방법

  ### 1) 토큰화 (Tokenization)

  분석 텍스트 정제

In [4]:
from konlpy.tag import Mecab
mecab = Mecab(dicpath = 'C:\mecab\mecab-ko-dic')

nodes = [token for token in mecab.pos(doc) if token[1] in ['NNG','NNP']] #NNG, NNP를 스코어 계산 대상(노드)로 제한
print(nodes)
tokens = [token for token in mecab.pos(doc)] #탐색할 토큰 전체
print(tokens)

[('과기', 'NNG'), ('정통부', 'NNG'), ('유영민', 'NNP'), ('장관', 'NNG'), ('참석', 'NNG'), ('기념행사', 'NNG'), ('투입', 'NNG'), ('여종', 'NNG'), ('데이터', 'NNG'), ('구축', 'NNG'), ('민간', 'NNG'), ('클라우드', 'NNP'), ('외부', 'NNG'), ('연계', 'NNG'), ('체계', 'NNG'), ('개방', 'NNG'), ('강화', 'NNG'), ('데일리', 'NNP'), ('이재운', 'NNP'), ('기자', 'NNG'), ('국가', 'NNG'), ('차원', 'NNG'), ('빅', 'NNG'), ('데이터', 'NNG'), ('활용', 'NNG'), ('시대', 'NNG'), ('산업', 'NNG'), ('창출', 'NNG'), ('기존', 'NNG'), ('산업', 'NNG'), ('변화', 'NNG'), ('혁신', 'NNG'), ('장', 'NNG'), ('센터', 'NNG'), ('문', 'NNG'), ('분야', 'NNG'), ('데이터', 'NNG'), ('경제', 'NNG'), ('발전', 'NNG'), ('정부', 'NNG'), ('청사진', 'NNG'), ('현실', 'NNG'), ('구현', 'NNG'), ('계획', 'NNG'), ('과학', 'NNG'), ('기술', 'NNG'), ('정보', 'NNG'), ('통신부', 'NNG'), ('서울', 'NNP'), ('중구', 'NNP'), ('대한', 'NNP'), ('상공', 'NNG'), ('회의소', 'NNG'), ('데이터', 'NNG'), ('생태', 'NNG'), ('조성', 'NNG'), ('혁신', 'NNG'), ('성장', 'NNG'), ('기반', 'NNG'), ('마련', 'NNG'), ('빅', 'NNG'), ('데이터', 'NNG'), ('플랫', 'NNG'), ('폼', 'NNG'), ('센터', 'NNG'), ('출범식', 'NNG'

### 2) 그래프 생성 (weighted edge 계산)

In [6]:
import networkx as nx

def connect(nodes, tokens):
    window_size = 3

    edges = []
    for window_start in range(len(tokens) - window_size + 1):
        window = tokens[window_start : window_start + window_size]
        for i in range(window_size):
            for j in range(i + 1, window_size):
                if (window[i] in nodes) & (window[j] in nodes ):
                    edges.append([window[i], window[j]])

    return edges

graph = nx.diamond_graph()
graph.add_nodes_from(list(set(nodes)))
graph.add_edges_from(connect(nodes, tokens))
graph

### 3) 각 노드의 score계산

In [7]:
scores = nx.pagerank(graph)

rank = sorted(scores.items(), key=lambda x: x[1], reverse=True)
rank[:5]

[(('데이터', 'NNG'), 0.04365352067396046),
 (('센터', 'NNG'), 0.016097998103787304),
 (('한국', 'NNP'), 0.012317440898987744),
 (('경제', 'NNG'), 0.011753305034386085),
 (('정보', 'NNG'), 0.01076734293352092)]

### 4)핵심 단어 추출

## 실습3. TextRank 핵심 구 추출

### 1) 명사를 기준으로 구 추출

In [9]:
phrases = []
phrase = ''
for word in tokens:
    if word[1][0] =='N':
        phrase +=str(word[0])
        phrase += ' '
    else :
        if phrase != ' ':
            phrases.append(phrase.strip())
        phrase = ' '
phrases[:10]

['과기 정통부',
 '일 유영민 장관 등 참석',
 '기념행사',
 '년',
 '억 원 투입',
 '여종 데이터 구축 민간 클라우드',
 '외부 연계 체계',
 '개방',
 '강화',
 '데일리 이재운 기자']

### 2) 각 구의 Score 계산

In [12]:
vocabs = [(r[0][0], r[1]) for r in rank]

phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_scores = 0
    keyword = ''
    for word in phrases.split():
        if word in vocabs.keys():
            phrases_score += vocabs[word.strip()]
            

TypeError: 'int' object is not subscriptable